In [2]:
import json

jsonl_file_path = "/data/jaesung/llm_for_diabetes/src/trial3/diabetes_food_hub_script.jsonl"
json_file_path = "/data/jaesung/llm_for_diabetes/src/trial3/Dog-Instruct/my_document.json"

data = []

with open(jsonl_file_path, 'r', encoding='utf-8') as file:
    for line in file:
        record = json.loads(line)
        if "script" in record:
            data.append({"text": record['script']})

with open(json_file_path, 'w', encoding='utf-8') as file:
    json.dump(data, file, indent=4, ensure_ascii=False)

In [ ]:
import json
import openai

OPENAI_API_KEY = "" 
openai.api_key = OPENAI_API_KEY

original_json_path = "/data/jaesung/llm_for_diabetes/src/trial3/Dog-Instruct/my_document.json"  
dog_instruct_json_path = "/data/jaesung/llm_for_diabetes/src/trial3/Dog-Instruct/my_task_limited_creativity.json"
output_jsonl_path = "my_task_final.jsonl" 

with open(original_json_path, "r", encoding="utf-8") as f:
    original_data = json.load(f)

with open(dog_instruct_json_path, "r", encoding="utf-8") as f:
    dog_instruct_data = json.load(f)

client = openai.OpenAI()

def refine_text_with_gpt(original_text, dog_instruct_entry):    
    prompt = f"""
    Your task is to refine the given instruction, input, and output while ensuring they accurately reflect the original text. 
    Preserve the intent and structure of the Dog-Instruct JSON while filling in any missing details from the original text.

    **Original Text (from the original JSON):**
    {original_text}

    **Current Dog-Instruct JSON (before refinement):**
    - Instruction: {dog_instruct_entry["instruction"]}
    - Input: {dog_instruct_entry["input"]}
    - Output: {dog_instruct_entry["output"]}

    **Task Requirements:**
    1. Preserve the existing instruction, input, and output structure from the Dog-Instruct JSON as much as possible.
    2. Fill in any missing or incomplete details using the original text without changing the meaning.
    3. Ensure that the response remains factual and does not introduce hallucinated content.
    4. The instruction should remain clear and relevant to the input.
    5. The input should contain meaningful content derived from the original text.
    6. The output should be a well-formed response based on both the Dog-Instruct JSON and original text.

    **Response Format (MUST be a valid JSON object without any additional text):**
    ```json
    {{
      "instruction": "...",
      "input": "...",
      "output": "..."
    }}
    ```
    DO NOT include any other text, explanation, or formatting outside the JSON block.
    """

    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": "You are an AI assistant that refines structured JSON data."},
                  {"role": "user", "content": prompt}],
        temperature=0.2
    )

    response_text = response.choices[0].message.content.strip()
    if response_text.startswith("```json"):
        response_text = response_text[7:]  # ```json 제거
    if response_text.endswith("```"):
        response_text = response_text[:-3]  # ``` 제거
    
    # print(f"GPT-4o Response:\n{response_text}\n")  # 응답을 출력하여 디버깅

    try:
        return json.loads(response_text)  # JSON 변환 시도
    except json.JSONDecodeError:
        print("❌ JSONDecodeError 발생! API 응답이 올바른 JSON 형식이 아닙니다.")
        return None  # None을 반환하여 이후 코드에서 예외 처리 가능

from tqdm import tqdm

with open(output_jsonl_path, "a", encoding="utf-8") as f:
    for orig, dog in tqdm(zip(original_data[464+441:], dog_instruct_data[464+441:])):
        fixed_entry = refine_text_with_gpt(orig["text"], dog)
        f.write(json.dumps(fixed_entry, ensure_ascii=False) + "\n")

print(f"Refined JSONL file saved at {output_jsonl_path}.")
# error idx -463, 903, 904

0it [00:00, ?it/s]

421it [1:33:34, 13.34s/it]

Refined JSONL file saved at my_task_final.jsonl.
